In [100]:
import pandas as pd
from selenium import webdriver
from bs4 import BeautifulSoup
from selenium.webdriver.support.wait import WebDriverWait


from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import ElementNotInteractableException 
from selenium.webdriver.support import expected_conditions as EC


# Step 1: Create a session and load the page
epm = 'https://dunksandthrees.com/epm?season=2023'

#url_list = [url1,url2,url3,url4,url5]

def check_exists_by_xpath(driver, xpath):
    try:
        driver.find_element(By.XPATH, xpath)
    except NoSuchElementException:
        return False
    return True
#url_list = [cs,pullup]
def get_tables(url_list):
    data = []
    for url in url_list:
        print(url)
        driver = webdriver.Chrome()
        driver.get(url)
        w_path = '//*[@id="player-stats"]/div[2]/table/tbody/tr[480]'
        xpath = '//*[@id="player-stats"]/div[2]/table'
        #button_path = '/html/body/div/main/div/div[3]/div[2]/table/thead/tr[2]/th[8]'
        '''
        try:
            elem = WebDriverWait(driver, 30).until(
                EC.presence_of_element_located((By.XPATH,xpath)))
            
        finally:
            driver.quit()
        
        '''
        #'''
        wait = WebDriverWait(driver, 10)
        result = wait.until(EC.presence_of_element_located((By.XPATH,w_path)))
        elem = driver.find_element(By.XPATH,xpath)
        
        # Wait for the page to fully load
        #print(elem.text)
        #elem = driver.find_element(By.XPATH,xpath)
        #driver.implicitly_wait(40)
        #driver.execute_script("window.scrollTo(0,document.body.scrollHeight)",elem)
        driver.implicitly_wait(40)
       
        '''if check_exists_by_xpath(driver, "//a[contains(text(),'>')]/preceding-sibling::a[1]"):
            number_of_pages = int(driver.find_element(By.XPATH, "//a[contains(text(),'>')]/preceding-sibling::a[1]").text)
            print(number_of_pages)'''
        # Step 2: Parse HTML code and grab tables with Beautiful Soup
        soup = BeautifulSoup(driver.page_source, 'html.parser')

        
        #return elem
        #rows = soup.find_all('tr')
        #print(rows)
        #return rows
        tables = soup.find_all('table')
        #vals = soup.find_all('td',class_ = 'player_name')
        #print(len(vals))
        #print(vals)
        # Step 3: Read tables with Pandas read_html()
        df = pd.read_html(str(tables))[0]
        #print(dfs)
        data.append(df)
        driver.close()
    return data
elem = get_tables([epm])

https://dunksandthrees.com/epm?season=2023


In [101]:
elem[0]

Unnamed: 0_level_0 Role                  Estimated +/-          \
                        PLAYER   GP       MPG    USG           OFF     DEF   
0                          NaN  NaN       NaN    NaN           NaN     NaN   
1          Nikola JokicDEN · C   28   33.1928  28%94        +7.399  +0.775   
2        Stephen CurryGSW · PG   26   34.4894  30%96        +7.599  +0.368   
3          Luka DoncicDAL · PG   30  36.61097  37%99       +7.7100  -0.159   
4    Anthony DavisLAL · C · GO   25   33.4836  28%94        +5.098  +2.296   
..                         ...  ...       ...    ...           ...     ...   
477         Joe InglesMIL · SF    3    16.449    14%          -4.8    -0.7   
478       Johnny DavisWAS · SG    9     5.448    12%          -4.6    -1.2   
479    Christian BraunDEN · SF   28   14.7411   11%7         -4.53   -2.82   
480                        NaN  NaN       NaN    NaN           NaN     NaN   
481                     PLAYER   GP       MPG    USG           OFF     DEF   

             Unnamed: 7_level_0 Shot Eff         ... Shot Location         \
         EPM                 eW      TS%    eFG  ...           Mid    3PT   
0        NaN                NaN      NaN    NaN  ...           NaN    NaN   
1    +7.9100              6.999    69%95  64%93  ...        63%100  33%26   
2     +7.899              6.699    68%95  63%89  ...         48%79  43%94   
3     +7.699             7.9100    61%72  56%66  ...         49%83  35%44   
4     +7.299              5.798    67%92  60%82  ...         44%52    29%   
..       ...                ...      ...    ...  ...           ...    ...   
477     -5.5               -0.1      14%    14%  ...           NaN     9%   
478     -5.8               -0.1      38%    27%  ...           25%    NaN   
479    -7.30              -1.21    51%13  49%23  ...           31%    31%   
480      NaN                NaN      NaN    NaN  ...           NaN    NaN   
481      EPM                 eW      TS%    eFG  ...           Mid    3PT   

    Unnamed: 13_level_0 Rebounds        Handle        Defense          \
                    FT%      OR%    DR%     A%    TO%     ST%     BL%   
0                   NaN      NaN    NaN    NaN    NaN     NaN     NaN   
1                 79%55     8%80  31%99  40%98  17%17  2.2%86  1.8%63   
2                 92%96     2%21  18%75  29%93  12%52  1.4%51  0.9%31   
3                 74%32     3%31  25%93  46%99  12%54  2.3%87  1.4%53   
4                 83%66    10%88  28%97  13%58   8%88  1.8%76  5.4%94   
..                  ...      ...    ...    ...    ...     ...     ...   
477                 NaN       5%    NaN    10%    31%    1.0%     NaN   
478                100%       2%    13%     7%     8%     NaN    2.6%   
479                 55%     5%63  12%39    5%9   9%82  1.1%26  1.1%41   
480                 NaN      NaN    NaN    NaN    NaN     NaN     NaN   
481                 FT%      OR%    DR%     A%    TO%     ST%     BL%   

    Unnamed: 20_level_0  
                     RK  
0                   NaN  
1                     1  
2                     2  
3                     3  
4                     4  
..                  ...  
477                 NaN  
478                 NaN  
479                 434  
480                 NaN  
481                  RK  

[482 rows x 21 columns]

In [26]:
df = elem[0]
df.columns = df.columns.droplevel()
df = df.dropna(subset = 'PLAYER')
df = df[df.PLAYER != 'PLAYER']

In [28]:
df.to_csv('epm_temp.csv',index = False)

In [95]:
df = pd.read_csv('epm_temp.csv')
df

,PLAYER,GP,MPG,USG,OFF,DEF,EPM,eW,TS%,eFG,...,Mid,3PT,FT%,OR%,DR%,A%,TO%,ST%,BL%,RK
0,Nikola JokicDEN · C,28,33.19280,28%94,7.399,0.775,7.910,6.999,69%95,64%93,...,63%100,33%26,79%55,8%80,31%99,40%98,17%17,2.2%86,1.8%63,1.0
1,Stephen CurryGSW · PG,26,34.48940,30%96,7.599,0.368,7.899,6.699,68%95,63%89,...,48%79,43%94,92%96,2%21,18%75,29%93,12%52,1.4%51,0.9%31,2.0
2,Luka DoncicDAL · PG,30,36.61097,37%99,7.710,-0.159,7.699,7.910,61%72,56%66,...,49%83,35%44,74%32,3%31,25%93,46%99,12%54,2.3%87,1.4%53,3.0
3,Anthony DavisLAL · C · GO,25,33.48360,28%94,5.098,2.296,7.299,5.798,67%92,60%82,...,44%52,29%,83%66,10%88,28%97,13%58,8%88,1.8%76,5.4%94,4.0
4,Joel EmbiidPHI · C,23,35.98250,37%99,5.098,1.994,6.999,5.597,64%86,55%62,...,47%75,34%36,86%82,6%68,26%94,25%88,13%44,1.6%66,4.3%91,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
474,Tyrese MartinATL · SF,10,3.83800,19%,-3.200,-1.700,-4.900,-0.100,47%,43%,...,50%,20%,100%,5%,12%,NaN,6%,1.2%,NaN,NaN
475,Udonis HaslemMIA · C,5,7.03500,13%,-4.600,-0.300,-5.000,0.000,14%,10%,...,20%,NaN,50%,8%,13%,NaN,NaN,1.3%,2.5%,NaN
476,Joe InglesMIL · SF,3,16.44900,14%,-4.800,-0.700,-5.500,-0.100,14%,14%,...,NaN,9%,NaN,5%,NaN,10%,31%,1.0%,NaN,NaN
477,Johnny DavisWAS · SG,9,5.44800,12%,-4.600,-1.200,-5.800,-0.100,38%,27%,...,25%,NaN,100%,2%,13%,7%,8%,NaN,2.6%,NaN


In [96]:

new_columns = []
columns = df.columns
for col in columns:
   # print(col)
    #print(df[col].dtype)
    if df[col].dtype == 'object':
       
        if '%' in df.loc[1][col]:
            df[[col,col+'_per']] = df[col].str.split("%",expand=True)
            df[col] = pd.to_numeric(df[col], errors='coerce')
            df[col+'_per'] = pd.to_numeric(df[col+'_per'], errors='coerce')
#df[['PLAYER','POS']] = df['PLAYER'].str.split('·',expand = True)
#df['PLAYER'] = df['PLAYER'].str.replace(' ', '')
df['pos'] = df['PLAYER'].str.split("·",expand = True)[1]

df['PLAYER'] = df['PLAYER'].str.split("·",expand = True)[0]
df['team'] = df['PLAYER'].str[-4:-1]
df['PLAYER'] =  df['PLAYER'].str[:-4]
df = df.round(2)

In [98]:
df.columns = [x.lower() for x in df.columns]
df

,player,gp,mpg,usg,off,def,epm,ew,ts%,efg,...,3pt_per,ft%_per,or%_per,dr%_per,a%_per,to%_per,st%_per,bl%_per,pos,team
0,Nikola Jokic,28,33.19,28,7.40,0.78,7.91,7.00,69.0,64.0,...,26.0,55.0,80.0,99.0,98.0,17.0,86.0,63.0,C,DEN
1,Stephen Curry,26,34.49,30,7.60,0.37,7.90,6.70,68.0,63.0,...,94.0,96.0,21.0,75.0,93.0,52.0,51.0,31.0,PG,GSW
2,Luka Doncic,30,36.61,37,7.71,-0.16,7.70,7.91,61.0,56.0,...,44.0,32.0,31.0,93.0,99.0,54.0,87.0,53.0,PG,DAL
3,Anthony Davis,25,33.48,28,5.10,2.30,7.30,5.80,67.0,60.0,...,NaN,66.0,88.0,97.0,58.0,88.0,76.0,94.0,C,LAL
4,Joel Embiid,23,35.98,37,5.10,1.99,7.00,5.60,64.0,55.0,...,36.0,82.0,68.0,94.0,88.0,44.0,66.0,91.0,C,PHI
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
474,Tyrese Martin,10,3.84,19,-3.20,-1.70,-4.90,-0.10,47.0,43.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,SF,ATL
475,Udonis Haslem,5,7.04,13,-4.60,-0.30,-5.00,0.00,14.0,10.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,C,MIA
476,Joe Ingles,3,16.45,14,-4.80,-0.70,-5.50,-0.10,14.0,14.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,SF,MIL
477,Johnny Davis,9,5.45,12,-4.60,-1.20,-5.80,-0.10,38.0,27.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,SG,WAS


In [99]:
df.to_csv('epm.csv',index = False)

In [72]:
df['team']

0      EN 
1      SW 
2      AL 
3      AL 
4      HI 
      ... 
474    TL 
475    IA 
476    IL 
477    AS 
478    EN 
Name: team, Length: 479, dtype: object

In [43]:
columns = df.columns
for col in columns:
    print(col)
    print(df[col].dtype)

PLAYER
object
GP
int64
MPG
float64
USG
int64
OFF
float64
DEF
float64
EPM
float64
eW
float64
TS%
float64
eFG
float64
Rim
float64
Mid
float64
3PT
float64
FT%
float64
OR%
float64
DR%
float64
A%
float64
TO%
float64
ST%
float64
BL%
float64
RK
float64
USG_per
float64
TS%_per
float64
eFG_per
float64
Rim_per
float64
Mid_per
float64
3PT_per
float64
FT%_per
float64
OR%_per
float64
DR%_per
float64
A%_per
float64
TO%_per
float64
ST%_per
float64
BL%_per
float64


In [ ]:
rows = driver.find_element("xpath",'//*[@id="player-stats"]/div[2]/table/tbody/tr[98]')

In [ ]:
rows.text

In [ ]:
//*[@id="player-stats"]/div[2]/table/tbody/tr[2]
//*[@id="player-stats"]/div[2]/table/tbody/tr[480]

In [ ]:
import io 
output = io.StringIO()
output.write(elem.text)

In [ ]:
pd.read_csv(output)